In [36]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import plotly.graph_objects as go
import numpy as np

load the dataset:

In [37]:
df_stl = pd.read_csv("parallel_with_c++_threads.csv")
df_stl_pinned = pd.read_csv("parallel_with_pinning.csv")
df_fastflow = pd.read_csv("parallel_with_fastflow.csv")
df_fastflow_reduce = pd.read_csv("parallel_with_fastflow_reduce.csv")

### Maximum speedup

From the sequential result, we know that:

In [38]:
NON_SERIAL_TIME = 204189058 #usec
SERIAL_TIME = 450435 #usec
ENTIRE_PROGRAM_TIME = 204639494 #usec
PERC_SERIAL = 0.220/100

Notice that also the creation of threads should go in the SERIAL_TIME. So, the serial time indicated above is just a "lower bound".

Given the PERC_SERIAL, we can compute the maximum speedup using the Amdhal law: 

In [39]:
MAX_SPEEDUP = 1/PERC_SERIAL
MAX_SPEEDUP

454.5454545454545

So, we can reach a maximum speedup of 454.

### Latency

In [40]:
def add_t_ideal(dataframe):
    dataframe['t_ideal'] = SERIAL_TIME + NON_SERIAL_TIME/dataframe['num workers']

add_t_ideal(df_stl)
add_t_ideal(df_stl_pinned)
add_t_ideal(df_fastflow)
add_t_ideal(df_fastflow_reduce)

In [41]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_stl['num workers'], y=df_stl['entire program time']*1e-6,
                    mode='lines',
                    name='stl'))
fig.add_trace(go.Scatter(x=df_stl_pinned['num workers'], y=df_stl_pinned['entire program time']*1e-6,
                    mode='lines',
                    name='stl_pinned'))
fig.add_trace(go.Scatter(x=df_fastflow['num workers'], y=df_fastflow['entire program time']*1e-6,
                    mode='lines',
                    name='fastflow'))
fig.add_trace(go.Scatter(x=df_fastflow_reduce['num workers'], y=df_fastflow_reduce['entire program time']*1e-6,
                    mode='lines',
                    name='fastflow reduce'))
fig.add_trace(go.Scatter(x=df_fastflow['num workers'], y=df_fastflow['t_ideal']*1e-6,
                    mode='lines',
                    name='ideal'))

fig.update_layout(title='Latency',
                   xaxis_title='n_w',
                   yaxis_title='Latency (in seconds)')

fig.show()

In [42]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_stl['num workers'], y=df_stl['entire program time']*1e-6,
                     name='stl'))
fig.add_trace(go.Bar(x=df_stl_pinned['num workers'], y=df_stl_pinned['entire program time']*1e-6,
                     name='stl_pinned'))
fig.add_trace(go.Bar(x=df_fastflow['num workers'], y=df_fastflow['entire program time']*1e-6,
                     name='fastflow'))
fig.add_trace(go.Bar(x=df_fastflow_reduce['num workers'], y=df_fastflow_reduce['entire program time']*1e-6,
                     name='fastflow reduce'))
fig.add_trace(go.Bar(x=df_fastflow['num workers'], y=df_fastflow['t_ideal']*1e-6,
                     name='ideal'))

fig.update_layout(title='Latency',
                  xaxis_title='n_w',
                  yaxis_title='Latency')

fig.update_layout(title='Latency',
                  barmode='group',
                  xaxis_title='n_w',
                  yaxis_title='Latency (in seconds)')

fig.show()


### READ + WRITE + Some overhead time

In [43]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_stl['num workers'], y=df_stl['serial time'] *1e-6,
                    mode='lines',
                    name='stl'))
fig.add_trace(go.Scatter(x=df_stl_pinned['num workers'], y=df_stl_pinned['serial time'] *1e-6,
                    mode='lines',
                    name='stl_pinned'))
fig.add_trace(go.Scatter(x=df_fastflow['num workers'], y=df_fastflow['serial time'] *1e-6,
                    mode='lines',
                    name='fastflow'))
fig.add_trace(go.Scatter(x=df_fastflow_reduce['num workers'], y=df_fastflow['serial time'] *1e-6,
                    mode='lines',
                    name='fastflow reduce'))

fig.update_layout(title='Serial time',
                   xaxis_title='n_w',
                   yaxis_title='Serial time')

fig.show()

### % Serial Fraction

In [44]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_stl['num workers'], y=df_stl['% serial'],
                    mode='lines',
                    name='stl'))
fig.add_trace(go.Scatter(x=df_stl_pinned['num workers'], y=df_stl_pinned['% serial'],
                    mode='lines',
                    name='stl_pinned'))
fig.add_trace(go.Scatter(x=df_fastflow['num workers'], y=df_fastflow['% serial'],
                    mode='lines',
                    name='fastflow'))
fig.add_trace(go.Scatter(x=df_fastflow_reduce['num workers'], y=df_fastflow_reduce['% serial'] ,
                    mode='lines',
                    name='fastflow reduce'))

fig.update_layout(title='% serial time',
                   xaxis_title='n_w',
                   yaxis_title='% serial time')

fig.show()

### Speedup

Speedup is T_seq/T_par(n). We define a function which add the attribute "speedup" to each dataframe:

In [45]:
def add_speedup(dataframe, T_seq):
    dataframe['speedup'] = T_seq/dataframe['entire program time']

add_speedup(df_stl, ENTIRE_PROGRAM_TIME)
add_speedup(df_stl_pinned, ENTIRE_PROGRAM_TIME)
add_speedup(df_fastflow, ENTIRE_PROGRAM_TIME)
add_speedup(df_fastflow_reduce, ENTIRE_PROGRAM_TIME)

Then we plot the speedups:

In [46]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_stl['num workers'], y=df_stl['speedup'],
                    mode='lines',
                    name='stl'))
fig.add_trace(go.Scatter(x=df_stl_pinned['num workers'], y=df_stl_pinned['speedup'],
                    mode='lines',
                    name='stl_pinning'))
fig.add_trace(go.Scatter(x=df_fastflow['num workers'], y=df_fastflow['speedup'],
                    mode='lines',
                    name='fastflow'))
fig.add_trace(go.Scatter(x=df_fastflow_reduce['num workers'], y=df_fastflow_reduce['speedup'],
                    mode='lines',
                    name='fastflow reduce'))
fig.add_trace(go.Scatter(x=np.arange(1,256,2), y=np.arange(1,256,2),
                    mode='lines',
                    name='ideal'))

fig.update_layout(title='Speedup',
                   xaxis_title='n_w',
                   yaxis_title='Speedup')

fig.show()

### Scalability

Scalability is T_par(1)/T_par(n):

In [47]:
def add_scalability(dataframe):
    T_par1 = dataframe[dataframe['num workers'] == 1]['entire program time'][0]
    dataframe['scalability'] = T_par1/dataframe['entire program time']

add_scalability(df_stl)
add_scalability(df_stl_pinned)
add_scalability(df_fastflow)

In [48]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_stl['num workers'], y=df_stl['scalability'],
                    mode='lines',
                    name='stl'))
fig.add_trace(go.Scatter(x=df_stl_pinned['num workers'], y=df_stl_pinned['scalability'],
                    mode='lines',
                    name='stl_pinning'))
fig.add_trace(go.Scatter(x=df_fastflow['num workers'], y=df_fastflow['scalability'],
                    mode='lines',
                    name='fastflow'))
fig.add_trace(go.Scatter(x=np.arange(1,256,2), y=np.arange(1,256,2),
                    mode='lines',
                    name='ideal'))

fig.update_layout(title='Scalability',
                   xaxis_title='n_w',
                   yaxis_title='Scalability')

fig.show()

- pinning threads is effective especially at the beginning. With 63 threads, we have efficiecy of 48 in stl_pinning and 37 in stl. In the end performances are the same.
- pinning threads make the curve more unstable. 
- every interval of 63 (approximately number of cores in the machine) we have an important decrease in efficiency/speedup for fastflow and stl_pinning. We can see this also in the % serial plot. Pinning threads cause this? Probably different pinning strategy may be better. 

### Efficiency

In [49]:
def add_efficiency(dataframe):
    dataframe['efficiency'] = dataframe['speedup']/dataframe['num workers'] * 100

add_efficiency(df_stl)
add_efficiency(df_stl_pinned)
add_efficiency(df_fastflow)
add_efficiency(df_fastflow_reduce)

In [50]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_stl['num workers'], y=df_stl['efficiency'],
                    mode='lines',
                    name='stl'))
fig.add_trace(go.Scatter(x=df_stl_pinned['num workers'], y=df_stl_pinned['efficiency'],
                    mode='lines',
                    name='stl_pinning'))
fig.add_trace(go.Scatter(x=df_fastflow['num workers'], y=df_fastflow['efficiency'],
                    mode='lines',
                    name='fastflow'))
fig.add_trace(go.Scatter(x=df_fastflow_reduce['num workers'], y=df_fastflow_reduce['efficiency'],
                    mode='lines',
                    name='fastflow reduce'))
fig.add_trace(go.Scatter(x=np.arange(1,256,2), y=np.ones((256))*100,
                    mode='lines',
                    name='ideal'))

fig.update_layout(title='Efficiency',
                   xaxis_title='n_w',
                   yaxis_title='Efficiency')

fig.show()